# Challenge Six: FEMA ReadyNow! Agent

In [1]:
import os
import logging
import vertexai
from pathlib import Path
from dotenv import load_dotenv
from utils.agent_callbacks import *
from utils.agent_tools import get_weather

env_path = Path.cwd().parent / '.env'
load_dotenv(env_path)

logger = logging.getLogger(__name__)
logging.basicConfig(filename="ReadyNow.log", level=logging.INFO)

GCP_PROJECT = os.getenv("GCP_PROJECT")
GCP_REGION = os.getenv("GCP_REGION")
GCP_BUCKET = os.getenv("GCP_BUCKET")
GOOGLE_MAP_KEY = os.getenv("GOOGLE_MAP_KEY")

vertexai.init(project=GCP_PROJECT, location=GCP_REGION, staging_bucket=GCP_BUCKET)

In [4]:
from google.adk.agents import LlmAgent, Agent, SequentialAgent
from google.adk.tools import google_search
from vertexai.preview import reasoning_engines
from IPython.display import Markdown, display

weather_agent = LlmAgent(
    name="weather_agent_v2",
    model="gemini-2.0-flash", # Can be a string for Gemini or a LiteLlm object
    description="Provides weather information for specific cities.",
    instruction="You are a helpful weather assistant. "
                "When the user asks for the weather in a specific city, "
                "use the 'get_weather' tool to find the information. "
                "If the tool returns an error, inform the user politely. "
                "If the tool is successful, present the weather report clearly as a markdown table.",
    tools=[get_weather], 
    output_key="weather_output",
    before_model_callback=chained_before_callback,
    after_model_callback=model_response_log_callback
)

search_agent = LlmAgent(
    name="search_agent",
    model="gemini-2.0-flash",
    instruction="Use the Google Search tool to find information about the user question in the state key 'user_query' from the context that you are a FEMA agent trying to determine news and emergency information.",
    tools=[google_search],
    output_key="search_results",
)

critique_agent = LlmAgent(
    name="critique_agent",
    model="gemini-2.0-flash",
    instruction="Critique the search results stored in the state key 'search_results'. Your purpose is to create better search results based on the fact that you are trying to determine emergency/disaster news and evacuation plans.",
    output_key="critique_results",
)

refine_agent = LlmAgent(
    name="refine_agent",
    model="gemini-2.5-flash",
    instruction="Refine the search results based on the critique stored in the state key 'critique_results' using the Google Search tool",
    tools=[google_search],
    output_key="refined_results",
)

presentation_agent = LlmAgent(
    name="presentation_agent",
    model="gemini-2.0-flash",
    instruction="Take the state keys 'refined_results' and 'weather_output' and format it so that it can be used as the final output to an end user. Be sure to include two days of weather results as part of your final output. Use markdown format.",
)

greeter_agent = Agent(
    name="greeter_agent",
    model="gemini-2.0-flash",
    instruction="You are a FEMA emergency response agent. Take the user query, then pass to 'search_agent'.",
    output_key="user_query"
)

search_sequence = SequentialAgent(
    name="search_sequence",
    sub_agents=[greeter_agent, weather_agent, search_agent, critique_agent, refine_agent, presentation_agent],
)

In [5]:
app = reasoning_engines.AdkApp(
    agent=search_sequence,
    enable_tracing=False,
)

user_id = "test-user-id"
session = app.create_session(user_id=user_id)

In [7]:
import warnings
warnings.filterwarnings('ignore')

for event in app.stream_query(
    user_id=user_id,
    session_id = session.id,
    message="I'm in Kerr County, Texas and flooding is happening please help me.",
):
  lastevent = event

display(Markdown(lastevent["content"]["parts"][0]["text"]))



/workspaces/genai-skills-workshop-shawn-mittal/venv/lib/python3.12/site-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


Okay, here is the final output, formatted for presentation to the user. I have included the weather forecast for today and tomorrow, and emphasized the urgent actions. Please be safe!

# Kerr County, Texas - Flood Emergency - IMMEDIATE ASSISTANCE

**If you are in immediate danger, call 911.**

## Situation Overview:

The situation in Kerr County is critical. As of July 9, 2025, the death toll from the flooding stands at 95, including 36 children. Sadly, 161 people are still missing in the county, including five campers and one counselor from Camp Mystic. The county is under a local state of disaster.

## Urgent Actions:

*   **Avoid Travel:** Residents are strongly encouraged to shelter in place and avoid travel.
*   **"Turn Around, Don't Drown":** Do not drive through floodwaters.
*   **Report Missing Persons:** If you have information about someone missing, contact authorities or update information at kerrvillemissing.dps.texas.gov or call 830-258-1111.
*   **Avoid Cleanup/Sightseeing:** Do not clear debris piles until inspected by search teams.
*   **Sign up NOW for CodeRed alerts** via the Kerr County Sheriff's Office website (Click HERE to sign up for free!). This system can notify you of emergencies, but some residents have reported delays in receiving alerts. In addition to signing up, consider alternative communication methods such as a NOAA weather radio or checking in with neighbors regularly.
*   **Stay Informed:** Monitor local news, weather alerts, and official social media channels for updates and instructions.
*   **Be cautious of potential scams.** Verify the legitimacy of any organization or individual requesting donations or assistance before providing any personal information or money.
*   **Emergency situations are constantly evolving.** Verify all information with official sources like local news, emergency management agencies, and government websites before making decisions.

## Emergency Contacts:

*   **Emergency:** 911
*   **Kerr County Sheriff's Office:** 830-896-1216
*   **Kerrville Police Department:** 830-257-8181
*   **Kerrville Fire/EMS:** 830-257-8181
*   **Poison Control:** 1-800-222-1222
*   **Suicide Prevention Hotline:** 1-800-SUICIDE or 1-800-784-2433

## Shelters:

*   First United Methodist Church: 321 Thompson Drive, Kerrville
*   Kerrville Salvation Army: 855 Hayes St. Kerrville, or call 830-465-4797. (Official in-kind donations partner)
*   Calvary Temple Church: 3000 Loop 534, Kerrville
*   Notre Dame Catholic Church: 929 Main Street, Kerrville
*   Schreiner University Event Center: 2100 Memorial Boulevard, Kerrville
*   Ingram Elementary School: 125 Brave Run in Ingram
*   Arcadia Live Theater: 717 Water Street in Kerrville
*   Comfort High School: 143 US-87 North, Comfort

## Weather Forecast:

Here's the weather forecast for the next two days. Note the low probability of precipitation, but be aware that conditions can change rapidly in a flooding situation.

| Period          | Short Forecast                       | Temperature | Wind                                  |
|-----------------|----------------------------------------|-------------|---------------------------------------|
| This Afternoon  | Sunny                                  | 88°F        | S wind around 15 mph, gusts up to 25 mph |
| Tonight         | Mostly Clear                           | 70°F        | SSE wind 10 to 15 mph, gusts up to 20 mph |
| Friday          | Mostly Sunny                           | 88°F        | SSE wind 10 to 15 mph, gusts up to 25 mph |
| Friday Night    | Partly Cloudy                          | 71°F        | SSE wind 10 to 15 mph, gusts up to 25 mph |

## Key Resources:

*   **State of Texas Emergency Assistance Registry (STEAR):** If you have a disability or need extra medical assistance during an emergency, register online at STEAR.TDEM.TEXAS.GOV, dial 2-1-1, or call 877-541-7905.
*   **DriveTexas.org:** Get up-to-the-minute road conditions.
*   **Real-Time Flood Data:** The Upper Guadalupe River Authority (UGRA) provides real-time data from USGS river flow gauging stations.
*   **Kerrville Public Utility Board (KPUB) for Power Outages:** 830-257-3050, or report outages 24/7 online through your SmartHub account, the SmartHub App, or by calling 830-257-2883.

## Other Assistance:

*   **Kerrville Salvation Army:** Contact for assistance at 855 Hayes St. or 830-465-4797.
*   **Kerr County Flood Relief Fund:** Managed by The Community Foundation of the Texas Hill Country.
*   **Texas 2-1-1:** Dial 2-1-1 or 877-541-7905 for resources including food, health, and housing.
*   **Texas Department of Insurance (TDI):** 800-252-3439 - For questions about insurance.
*   **FEMA Disaster Assistance:** disasterassistance.gov or 800-621-3362.

## Resources for Vulnerable Populations:

*   Hill Country MHDD Centers: 830-258-5400
*   Alamo Area Council of Governments (AACOG): 210-362-5229
*   "Take Me Home" Program (Kerrville Police Department): Register at https://www.frontlinepss.com/kerrville-tx by clicking on "At Risk Residents".
*   Human Services Annex: Contact Kerrville Director of General Operation for food, clothing, and mental health services.

## Pet Resources:

*   Kerrville Pets Alive!: Contact for pet resources and recovery at 830-200-0539 or info@kerrvilleptsalive.org. Pet supply donations can be delivered to 2102 Memorial Blvd in Kerrville.
*   Freeman-Fritts Animal Shelter: 515 Spur 100, Kerrville, Texas 78028. Phone: 830-257-4144.
*   Animal Welfare Society of Kerr County: Provides affordable vet care, low-cost spay/neuter, and adoptions.
*   Austin Pets Alive!: Is in Kerrville providing medical care and transporting animals.

## What to Do After the Flood:

*   Document Everything: Take photos/videos of damage *before* cleaning.
*   File Insurance Claims: Contact the National Flood Insurance Program (NFIP) at 877-336-2627.
*   Legal Aid: State Bar of Texas Legal Hotline: 800-504-7030 (for low-income Texans).
*   Mental Health Support: Disaster Distress Helpline: 800-985-5990 (24/7 support).

## Road Closures:

As of July 9, 2025:

*   State Highway 39: Closed from Highway 27 to FM 187.
*   FM 1340: Closed from SH 39 to State Highway 41.
*   Bear Creek Road: Closed from Sheppard Rees Road to Freedom Trail.
*   **Check DriveTexas.org for the most current road closures.**

## Important Considerations:

*   Be aware that flash flooding can occur along the Guadalupe River and other waterways.
*   The Kerrville Public Utility Board (KPUB) is working to restore power but faces challenges due to the damage.
*   Emergency responders are using heavy equipment for search and rescue. Please avoid these areas.

